# Repurposing MARC data for analysis #

\[This notebook dives right into a fairly involved exercise that draws on a prior discussion of some programmimg fundamentals like data structures, variables, conditionals, and loops. For a hands-on review of those fundamentals using MARC data for the examples, see notebooks A and B. It also presumes that you're already reasonably conversant with the nature of MARC data. Your folder of handouts includes a MARC cheat sheet that might come in handy.\]

MARC records are meant to provide information for presentation to users of library catalogs, but they include information that could be used to answer all sorts of potentialy interesting questions. In this example, I want to look at a group of MARC records from the English Short Title Catalogue associated with William Bowyer to see how many of the titles he printed were written by living authors and how many were re-publications of works by dead authors. We'll use a Python script to read our MARC records, perform some basic analysis, and then export the results to a .csv file that we can import into a program like Excel for graphing. (We'll also generate a primitive chart using Python, so we can see results right away.

At the heart of this analysis is a very simple mathematical comparison. To figure out whether an author was alive or dead when a title was published, we'll need to compare the publication year with the year of the author's death. If the author's death year is greater than the publication year, the author died after the book was published. If the death year is less than the publication year, the author died before the book was published. (If the author's death year is equal to the publication year, we have an ambiguous result: perhaps the book was published shortly before the author's death, perhaps it was published after the author's death in order to capitalize on revived notoriety. Without more precise dating, we can't be sure.)

But even this simple mathematical comparison requires some finagling.

## Publication year ##

Rather than relying on the date in the imprint statement (MARC 260|c), which can take various forms (e.g. roman numerals, prefixed with phrases like "Printed in the year," etc.), we'll look to the MARC 008 control field for the publication year. This is a fixed-length field that reports structured information about the record, with different character positions reserved for different pieces of information.

For this example, we're going to ignore some of the potential complexities that could come up when working with the 008 field. There can be different kinds of dates, for instance (single dates, date ranges, questionable dates, etc.), and the nature of the date(s) provided is communicated by character 6. For our purposes, we're going to treat the first date in the MARC 008 field as the publication date, plain and simple. Uncertainty about a date can also be communicated by giving a partial date with "u" in place of the uncertain digits ("169u" would mean some time in the 1690s, for instance). It just so happens that there are no "u"s in our sample set, but this problem could require further condition testing and logic if we were trying to do this kind of work on a larger set of MARC records.

## Author's dates ##

The author's dates are available in MARC 100|d, but present some challenges. Dates typically take the form of two four-digit numbers separated by a hyphen and followed by a period (e.g. "1660-1734.") But that's not always the case. Sometimes we know only the author's birth date (e.g., "b. 1660"), sometimes only the death date (e.g., "d. 1734"). Sometimes we don't know precise dates (e.g., "1660 or 61 - 1734" or "ca. 1660 - 1734" or "1720 - ca. 1778"), sometimes we only know the dates the author was active as a writer, but not the dates of his or her birth or death (e.g., "fl. 1763-1770"). So we'll need to do some work first just to determine whether a date is usable for our purposes.

Another issue is that, while we recognize the (many, variable) forms of information in MARC 100|d as years, the contents of the field are, from the computer's point of view, simply strings of texts rather than numbers that can be computed. We'll need to convert strings to integers in order to do the arithmetic to see if the author was living or dead in the year of publication.

## Dealing with uncertainties ##

Though we're going to disregard some nuances of our MARC data, we're not going to be totally reckless: for one reason or another, not all of the records we'll be analyzing will provide a clear answer to the question we're asking. The least we can do is keep track of how many records we've excluded from our analysis and gather some information about them so that we can scrutinize our preliminary results and identify potential issues we'd need to look at more closely. As we process these records, we'll keep track not just of the results we can classify as either living or dead, we'll also keep track of the records we can't do anything with: records for which there's no author given; records for which the author field doesn't have a date field; and records for which the available dates don't allow us to judge whether the author was living or dead.

## Techniques in this exercise ##

Even in this relatively simple case, we'll have to do several things:

* Conditional checks to learn about the contents of fields before we do anything with them.
* String operations (including regular expressions) to find and extract portions of the fields that we need.
* Data type conversion to turn strings into integers
* Math operations to compare different values
* Working with a variety of variables, including integers and dictionaries, to keep track of our results
* Exporting our results to a file.
* Representing (some of) our data in a simple chart.
 
## Notes ##

There are all sorts of caveats that go along with trying to use MARC records this way, and I could list several potential problems with the analysis in this example. One big one is the fact that the ESTC record isn't a uniform unit of measure: some authors may be over-counted in this exercise because there could be multiple ESTC records representing different cataloged variants of what are really the same edition. (I know for sure that happens with Thomson's *Sophonisba*, for instance.) So, while this exercise shows how to munge the data from MARC records, there are always going to be more substantive questions that you'll need to ask about the data you're looking at.

The code in this example was written to break things down into a series of relatively small steps and to prioritize clarity. If you've worked with Python before, you'll probably see things that could be done more elegantly and with fewer lines of code.    

## First, where are you running this notebook? ##

These notebooks were first written for use in L-100 (Digital Approaches to Bibliography and Book history) at Rare Book School in 2018. If you were a student in that course, you received an image of a virtual machine running Ubuntu Linux, along with the files that the various Jupyter Notebooks work with. If you've come to this notebook from a link at GitHub to MyBinder, you won't have those files, but I've added them to GitHub.

NOTE: If you're running this notebook from the RBSDigitalApproaches2018 virtual machine, you'll need to be sure that you're running `jupyter notebook` with the `marc` virtualenv activated. If you run into errors below related to missing libraries, quite the notebook and Jupyter Notebooks, go back to your terminalk and execute `workon marc` to activate that virtualenv. (If another virtualenv is active, you may have to execute `deactivate` first.) 

The first thing we need to do is to determine what file paths to use in the rest of the code below. Simply comment or uncomment the appropriate line by adding or deleting the octothorpe (#) at the beginning of the line, then run the cell.

In [ ]:
# If you're running this notebook from the RBSDigitalApproaches2018 virtual machine
# in VirtualBox, make sure the line below is uncommented (that's the default for this
# notebook.)
location = 'rbs_virtual_machine'

# If you're running this notebook at MyBinder using a link from the GitHub repository,
# comment out line 4, above, and uncomment line 8, below:
#location = 'mybinder'

if location == 'rbs_virtual_machine' :
    input_path = '/media/sf_RBSDigitalApproaches/data/0611_Tuesday_data/'
    output_path = '/media/sf_RBSDigitalApproaches/output/'
elif location == 'mybinder' :
    input_path = 'data/'
    output_path = 'output/'
    # If you're running this notebook at MyBinder.org, the code on this page will end up
    # writing files to the "output" folder. To download the files you create, go to the 
    # Jupyter Notebook "Home" tab for the repository, enter the "output" folder, click 
    # on the filename of the file you created, then click the "Download" button.

### Importing libraries ##

In addition to Pymarc (introduced in notebooks A and B), we're using two other Python packages: 
1. `re` allows us to work with regular expressions, allowing us to search for patterns of text, even if we don't know the precise text (e.g. four numerical characters, rather than "1713). 
2. `csv` allows us to read and write .csv (comma-separated value) files, which can, among other things, be opened by spreadsheet software like Excel.

In [ ]:
from pymarc import MARCReader
import re
import csv

### Input ###

In order to take advantage of Jupyter Notebook's capacity to present code in short, readable, interactive form, we're going to handle the opening and reading of our MARC file a little differently in this script than in the others. The other examples handle all of their processing inside a single "with" statement, but if there's a way to begin such a statement in one cell and continue it in subsequent cells, I haven't found it. So, instead, we will read the MARC data in at the beginning of our script and store the parts of it that interest us as a dictionary. Since this dictionary is created at the very outset of the script, it has "global scope" and will remain available to all subsequent cells.

In [ ]:
# Create an empty dictionary to hold selected data from our MARC records.
marc_data = {}

# Open our file of MARC records
with open(input_path + 'Bowyer_from_ESTC-full.mrc', 'r') as infile :
    # Pass the contents of the MARC file to Pymarc's MARCReader function
    reader = MARCReader(infile)
    
    # Read our MARC records one at a time, storing just the information we'll need for this script in a series
    # of variables
    for record in reader :
        estc_num = record['001'].data
        marc_008 = record['008'].data
        # We only need a portion of the MARC 008 controlfield. We're going to turn it into an integer now because
        # we'll need to do math with it later. We'll also use it as a key for dictionary entries, but dictionary
        # keys can be integers, so there's no reason not to just make it an integer now.
        pub_year = int(marc_008[7:11])
        
        # Not all records have a 100 field, but if we find one, we'll construct a dictionary to hold the
        # subfields we want. (Note that I've added the line breaks purely for legibility in Jupyter Notebook.
        # If you were using a code editor that supported line wrapping, you wouldn't need to put them there.)
        if record['100'] :
            marc_100 = {
                '100$a': record['100']['a'], 
                '100$c': record['100']['c'], 
                '100$q': record['100']['q'],
                '100$d': record['100']['d'],
            }
            # If we found a 100 field, we're ready to construct an entry in our marc_data dictionary. We'll
            # use the ESTC number as the key, then hold a series of values in a nested dictionary. Note that
            # the marc_100 variable we're assigning as the value of key 'marc_100' is, itself, a dictionary,
            # so that's a dictionary-within-a-dictionary-within-a-dictionary. It's like a data turducken.
            # We don't have a value for author_death yet, but will get it later.
            marc_data.setdefault(
                estc_num, 
                {'pub_year': pub_year, 
                 'marc_100': marc_100, 
                 'author_death': 0
                }
            )
        # If we didn't find a 100 field, we'll construct our dictionary entry a little differently, supplying a 
        # publication date, but setting the values of the other keys to None or 0.
        else :
            marc_data.setdefault(
                estc_num, 
                {'pub_year': pub_year, 
                 'marc_100': None, 
                 'author_death': 0
                }
            )
## Let's see what we have so far:
for estc_num in sorted(marc_data.keys()) :
    print(estc_num)
    print(marc_data[estc_num])

### Start keeping track of information year, by year ###

Now that we have the MARC data we need in our marc_data dictionary, we'll start building up information about the authors whose tests were published each year.

In [ ]:
# Create an empty dictionary to hold information about years
yearly = {}
# Work through the entries in our marc_data dictionary.
for estc_num in marc_data.keys() :
    # Retrieve the pub_year so we can use it to construct a series of keys for our new dictionary
    pub_year = marc_data[estc_num]['pub_year']
    # Use setdefault to create new entries in our yearly dictionary: the pub_year serves as the key, and we
    # create a dictionary with a bunch of keys whose values are all 0 at the moment
    yearly.setdefault(
        pub_year,
            {'total_records':0,
             'no_dates':0,
             'indeterminate':0,
             'ambiguous':0,
             'living_authors':0,
             'dead_authors':0,
             'no_author':0
            }
    )
    # We can now increment the count of total_records for this year
    yearly[pub_year]['total_records'] += 1

# Let's see what we just did.
for year in sorted(yearly.keys()) :
    print(year)
    print(yearly[year])

### What if there's no 100 field at all? ###

We can't very well decide if the author was alive or dead when the text was published if there wasn't an author. (In this case, we're saying a record without a 100 field had no author, which isn't strictly accurate. The record could have a 110 field for a corporate author, for instance, but for our purposes, at least, corporations aren't people.) We're not going to be able to do anything with those entries towards answering our question about living vs. dead authors, but we can still account for the reason we're leaving them out of our calculations. If nothing else, we'll be able to see what percentage of the total records could be determined to be by living vs. dead authors.

In [ ]:
for estc_num in marc_data.keys() :
    pub_year = marc_data[estc_num]['pub_year']
    if marc_data[estc_num]['marc_100'] is None :
        yearly[pub_year]['no_author'] += 1
        
# Let's see the effect this had on our yearly dictionary:
for year in sorted(yearly.keys()) :
    print(year)
    print(yearly[year])

### On the Death of the Author (with apologies to Roland Barthes) ###

Now on to the records that do have 100 fields. At least we know there's an author, but we have to do a few checks to see if we can find a usable death year for the author. 

There are a few regular expressions in this block. You can refer to your Regular Expressions crash course handout for a little more documentation to get you started. These can be very useful, but are a little strange at first. And they can get pretty complex and confusing sometimes, but these three are reasonably simple:

At line 21, we define a generic pattern for a year: 
* The square brackets say we're interested in a class of characters (in this case, numerical characters between between 0 and 9).
* The curly braces say how many of that class of characters we want to find in a row.
(This pattern could also be expressed as `[\d]{4}` (i.e., four digit characters). If we wanted, say, only years in the 1760s, we could say `176\d` or `176[0-9]`.

The pattern at line 27 is a little richer, and starts to show the flexibility and usefulness of regular expressions for some tasks.
1. The first thing to note about this pattern is the use of the vertical pipe character (`|`), which means "or"--we're interested in anything that matches any one of four possible patterns (based on my quick survey of the contents of the 100|d subfields)
2. The first pattern is the letters "fl", but only at the beginning of the string, and possibly followed by a period. 
    + The caret indicates that we want to search at the beginning of the string (we'd use a dollar sign to indicate the end of the string). Note, though, that the caret has a different meaning inside square brackets, which you'll likely need to be aware of in the future.
    + The period has a special meaning reserved for it in regular expressions (it stands for "any character"). So if we want to look for a literal ".", we need to "escape" the period so that it doesn't get interpreted as "any character." 
    + The question mark means that the character immediately preceding--the literal period (`\.`)--is optional: it may or may not be present and our pattern will still match. (I've made it optional because, every once in a great while, even catalogers at the ESTC make typographical errors and forget a punctuation mark.)
3. The second pattern is just like the first: letter "b" at the beginning of the string, optionally followed by a literal period.
4. The third pattern is the letters "or" surrounded by white spaces on either side. (`\s` is one of several special characters with reserved meanings in regular expressions--like `.` or the `\d` that I mentioned above.

See if you can figure out what the regular expression at line 30 means--it's the simplest of the bunch. (Though, actually, now that I look at it again, we could make it even simpler since we're using `match()` rather than `search()`, since match looks at the beginning of the string...)


In [ ]:
# Look through the entries in our marc_data dictionary
for estc_num in marc_data.keys() :
    # Get the pub_year value to use as a key to update the apropriate entry in our yearly dictionary
    pub_year = marc_data[estc_num]['pub_year']
    
    # Make sure there's a value for the 'marc_100' key
    if marc_data[estc_num]['marc_100'] is not None :
        author_data = marc_data[estc_num]['marc_100']
        
        # If no $d subfield is present, we can't make a determination about the death year, but we can 
        # note the number of records in each year for which no dates were available.
        if author_data['100$d'] is None :
            yearly[pub_year]['no_dates'] += 1
        
        # If a $d subfield *is* present, we have a few questions we have to ask before we know what to do
        else :
            # Get the contents of the 100$d subfield
            dates = author_data['100$d']
            # Construct a regular expression pattern to find four numerical digits in a row (to use later 
            # in two different scenarios.)
            year_test = re.compile('[0-9]{4}')  
            
            # If our date field begins with "fl" or "fl."; if it begins with "b" or "b."; if it contains
            # " or "; or if it contains "ca" or "ca." we're not going to be able to make a solid determination
            # about the author's death year. (Actually, if we were stubborn enough, we could probably code our 
            # way around this problem. But for now let's just call this one "indeterminate.")
            if re.search('^fl\.?|^b\.?|\sor\s|ca\.?', dates) :
                yearly[pub_year]['indeterminate'] += 1
            
            elif re.match('^d',dates) :
                # Perform a regular expresion findall() search for the pattern year_test (see line 21) in the 
                # string dates (see line 18). findall() returns a list of results. We want the first one--which 
                # should be the only one in this case: the one at index [0]. 
                death_year = re.findall(year_test, dates)[0]
                # Make death_year the value of 'author_death' in the entry for this estc_num in the marc_data
                # dictionary (we set a default value of 0 in an earlier cell). We want to be able to do math with
                # this number, rather than have it just be a string of four numerical characters, so we wrap
                # death_year with int() to turn it into an integer.
                marc_data[estc_num]['author_death'] = int(death_year)
            
            # Finally, in all other cases, we should be dealing with strings of digits separated by hyphens. 
            else :
                # Perform a regular expression findall() search for the year_test pattern in the dates string
                years = re.findall(year_test, dates)
                # Assuming we find years (it would be really weird if we didn't, but who knows?)...
                if years :
                    # findall() returns a list of results. In this case, we want the second string of four digits:
                    # the one at index[1]
                    death_year = years[1]
                    # Assign death_year as the value of the 'author_death' key for this estc_num's entry in the 
                    # marc_data dictionary. Turn it into an integer so we can do math with it.
                    marc_data[estc_num]['author_death'] = int(death_year)

### Progress check ###

Let's see what we just did: added some "indeterminate" results to our yearly count and supplied some `author_death` years to the entries in our `marc_data` dictionary.

In [ ]:
for year in sorted(yearly.keys()) :
    print(year)
    print(yearly[year])

In [ ]:
for estc_num in sorted(marc_data.keys()) :
    print(estc_num)
    print(marc_data[estc_num])

### Figure out who was alive and who was dead ###

We've now added all the information we need in each entry of our `marc_data` dictionary to decide whether an author was alive or dead when a title was published. (To the extent we can, that is. We've already registered many of the problem cases in our yearly dictionary: records with no authors; records whose authors have no dates; and our "indeterminate" dates ("b." "fl." and "ca.").

So now we're down to the cases where we can actually compare reliable years.

In [ ]:
for estc_num in marc_data.keys() :
    pub_year = marc_data[estc_num]['pub_year']
    author_death = marc_data[estc_num]['author_death']
    # Remember that the value for our default 'author_death' was 0. If we haven't been able to assign a death_year
    # for a record, that value's still 0. So we need to only work on records where author_death is not equal to 0.
    if author_death != 0 :
        # Okay, this should have been easy. But what if the author died in the same year the book was published?
        # We can't make a solid determination with more precise dates...  
        if author_death == pub_year :
            # So we increment that year's count of "ambiguous" publications, sigh heavily, and move on...
            yearly[pub_year]['ambiguous'] += 1
        # If the author_death is greater than the publication year, the author was still alive when the 
        # text was published. (Note that we're performing a mathematical comparison here. Good thing we turned
        # those strings into integers...)
        elif author_death > pub_year :
            # Increment this year's count of living_authors
            yearly[pub_year]['living_authors'] += 1
        # If author_death is less than pub_year...
        elif author_death < pub_year :
            # This is an ex-author
            yearly[pub_year]['dead_authors'] += 1

### Progress check ###
 
 Our yearly dictionary is now complete

In [ ]:
for year in sorted(yearly.keys()) :
    print(year)
    print(yearly[year])

### Writing our results to a file ###
Maybe we want to save this information so that we can work with it in a spreadsheet program like Excel...

In [ ]:
# Open a .csv file in writable binary mode. 

with open(output_path + 'Bowyer_by_year.csv', 'wb') as outfile1 :
    # This message is just here so we'll see that something's happening. Because this is going to be done
    # so quickly you'll miss it otherwise.
    print('Writing Bowyer_by_year.csv ...')
    
    # Provide a list of fieldnames to use in the first row of our .csv file
    fieldnames1 = ['Year','Living Authors','Dead Authors','No Author','No Dates','Indeterminate','Ambiguous',
                  'Total Records']
    
    # Initiate our csv DictWriter
    writer1 = csv.DictWriter(outfile1, delimiter=',', fieldnames=fieldnames1)
    # Write out the first row of fieldnames
    writer1.writeheader()
    
    # For each year in our yearly dictionary...
    for year in sorted(yearly.keys()) :
        # Construct a dictionary of keys corresponding to our fieldnames and values drawn from the yearly 
        # dictionary
        row1 = {
            'Year': year, 
            'No Author': yearly[year]['no_author'], 
            'No Dates': yearly[year]['no_dates'],
            'Indeterminate': yearly[year]['indeterminate'],
            'Ambiguous': yearly[year]['ambiguous'], 
            'Living Authors': yearly[year]['living_authors'],
            'Dead Authors': yearly[year]['dead_authors'], 
            'Total Records': yearly[year]['total_records']
        }
        # Write each row of results to our .csv file
        writer1.writerow(row1)
    # This message is here to let you know the file is complete. Because it will be done before you know it.
    print('Finished writing Bowyer_by_year.csv.')

### Or charting our results ###
With the proper pacakges installed, we can just visualize our results right here. I haven't done much work with matplotlib in Python, so I'm sure it's possible to make a much prettier chart, but here's a rough stab. Full documentation is at [https://matplotlib.org](https://matplotlib.org), and lots of snippets are available on StackOverflow and elsewhere.

\[NOTE: You may have to run this cell twice for the chart to appear.\]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set the size of the chart that will display below
plt.rcParams['figure.figsize'] = [15, 5]

# Create three empty lists to hold our years (used as labels) and our two data series:
years = []
living_ct = []
dead_ct = []

# Populate our empty lists with information from our yearly dictionary
for year in sorted(yearly.keys()) :
    years.append(year)
    living_ct.append(yearly[year]['living_authors'])
    dead_ct.append(yearly[year]['dead_authors'])

# Set up our graph with one entry on the x axis for each of our years and set the width of our bars
# so there's a little white space between them.
ind = np.arange(len(years))
width = .8 

# Define our two data series
p1 = plt.bar(ind, living_ct, width, color='#d62728')
p2 = plt.bar(ind, dead_ct, width, bottom=living_ct)

# Some formatting of the chart
plt.title('Living vs. dead authors')
plt.xticks(ind, years)
plt.xticks(rotation = 90)
plt.yticks(np.arange(0, 80, 10))
plt.legend((p1[0], p2[0]), ('Living', 'Dead'))

plt.show()